# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [3]:
import cPickle as pickle
import slate 
import re

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [296]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [295]:
cleaned_joined = joined.replace('.', ' ')

In [380]:
# cleaned_joined[int(len(cleaned_joined)*0.6):int(len(cleaned_joined)*0.7)]

In [375]:
with open ('joined_inspect', 'w') as f:
    f.write(cleaned_joined)

entries I'm interested in start at split doc 17

## RE for flavor compounds and ingredients

This is the ultimate regex pattern. It finds the natural occurence, and searches below for the next flavor compound
'Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+'

In [467]:
# sq = re.compile('Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+(.{0,5}[A-Z]{2,20}.*)\n+')
sq = re.compile('Natural occurrence: (.+)\n\n[\x0c]?(.*[A-Z]{4,20}.*)\s{2,4}(.*[A-Z]{4,20}.*)?\n?\n?')
raw_ingredients = re.findall(sq, cleaned_joined)

ingredient_series = pd.Series([i[1] for i in raw_ingredients])

In [463]:
len(raw_ingredients)

In [392]:
stop_strings = ingredient_series.value_counts()[:10].index[:4]

top_terms = list(ingredient_series.value_counts().index.values[:10])
stop_strings=top_terms[:4]+ top_terms[5:7] + [top_terms[8]]

In [330]:
# cleaned_ingredients = [comp.strip() for comp in raw_ingredients if comp not in stop_strings]
# cleaned_ingredients[:10]

In [455]:
with_three = [i for i in raw_ingredients if i[2]]

In [456]:
len(with_three)

In [469]:
ingredient_series.value_counts().value_counts()

## Here I have the material I need to build the ingredient:compound mapping

In [470]:
raw_ingredients